In [357]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import numpy as np
import pandas as pd
import glob
import cv2
import tensorflow as tf
from matplotlib import pyplot as plt 
from tensorflow.keras import layers


In [450]:
# import images as numpy datasets

train_path = 'data/nn_train'
im_list = sorted(glob.glob(train_path + '/*.png', recursive=True))

image = cv2.imread(im_list[0], cv2.IMREAD_GRAYSCALE)
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ]
)



train_x = []
for im_name in im_list:
    thename = im_name.split('\\')[-1]
    img = cv2.imread(im_name, cv2.IMREAD_GRAYSCALE)
    img = tf.expand_dims(img, 0)
    for i in range(10):
        augmented_image = data_augmentation(img)
        train_x.append(np.array(augmented_image[0]) / 255)


test_path = 'data/nn_test'
im_list = sorted(glob.glob(test_path + '/*.png', recursive=True))

test_x = []
for im_name in im_list:
    thename = im_name.split('\\')[-1]
    img = cv2.imread(im_name, cv2.IMREAD_GRAYSCALE)
    img_n = np.array(img)
#     img_n = np.ravel(img_n)
    test_x.append(img_n / 255)



In [338]:
np.shape(train_x)

(7500, 64, 32)

In [339]:
ids_train = pd.read_csv('data/ears/annotations/recognition/ids_train.csv', sep=',', header=None)
ids_test = pd.read_csv('data/ears/annotations/recognition/ids_test.csv', sep=',', header=None)

ids_train = ids_train.rename(columns={0: 'path', 1: 'class'})
ids_test = ids_test.rename(columns={0: 'path', 1: 'class'})

len(ids_train['class'].unique())

100

In [340]:
train_y = np.reshape(np.array(ids_train['class']), (750, 1))
test_y = np.reshape(np.array(ids_test['class']), (250, 1))

train_y = np.repeat(train_y, 10)
np.shape(train_y)

(7500,)

In [377]:
y_one_hot = train_y.reshape(1,-1)

y_one_hot = tf.one_hot(train_y, depth=100)
y_one_hot = np.reshape(y_one_hot, (7500, 100))
train_x = np.array(train_x)
test_x = np.array(test_x)

In [401]:
train_x = np.reshape(train_x, (7500, 96, 96, 1))

In [402]:
np.shape(train_x)

(7500, 96, 96, 1)

In [444]:

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu', padding='same', input_shape=(96,96,1)))
model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.1))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(units=1500, activation='relu'))
model.add(Dense(units=1500, activation='relu'))
model.add(Dense(units=100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [445]:
model.summary()

Model: "sequential_207"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_354 (Conv2D)         (None, 96, 96, 32)        544       
                                                                 
 conv2d_355 (Conv2D)         (None, 96, 96, 32)        16416     
                                                                 
 max_pooling2d_220 (MaxPooli  (None, 48, 48, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_46 (Dropout)        (None, 48, 48, 32)        0         
                                                                 
 conv2d_356 (Conv2D)         (None, 48, 48, 32)        9248      
                                                                 
 conv2d_357 (Conv2D)         (None, 48, 48, 32)        9248      
                                                    

In [447]:
model.fit(train_x, y_one_hot, epochs=5, batch_size=75)

Epoch 1/5
100/100 [==============================] - 223s 2s/step - loss: 14.6833 - accuracy: 0.0099
Epoch 2/5
100/100 [==============================] - 256s 3s/step - loss: 14.6227 - accuracy: 0.0133
Epoch 3/5
 58/100 [================>.............] - ETA: 1:38 - loss: 14.5971 - accuracy: 0.0143

KeyboardInterrupt: 

In [426]:
np.shape(train_x)

(7500, 96, 96, 1)

In [427]:
_, accuracy = model.evaluate(train_x, y_one_hot)
accuracy

235/235 [==============================] - 44s 187ms/step - loss: 3.7798 - accuracy: 0.1791


0.17906667292118073

In [432]:
predictions_probs = model.predict(test_x)
predictions = predictions_probs.argmax(axis=-1)

# for pr in predictions_probs:
#     predictions.append(np.argmax(pr))
    

# test_y = np.reshape(test_y, (250,))

In [439]:
np.argmax(predictions_probs[0])

73

In [448]:
# classification_accuracy = np.sum(predictions == test_y)/len(predictions)
# classification_accuracy
pred_df = pd.DataFrame({'predicted': predictions, 'truth': test_y})
pred_df['correct'] = pred_df['predicted'] == pred_df['truth']
print('Classification accuracy: ' + "%0.2f" % ca )

Classification accuracy: 2.28


In [449]:
np.array([1, 1, 1, 1, 1]) / 10

array([0.1, 0.1, 0.1, 0.1, 0.1])

In [435]:
pred_df

,predicted,truth,correct
0,73,11,False
1,7,11,False
2,8,11,False
3,51,1,False
4,3,12,False
...,...,...,...
245,7,100,False
246,74,100,False
247,7,100,False
248,2,93,False
